A context manager is a class that implements `__enter__()` and `__exit__()`

The context manager object will be used in `with` statement.
```python
with context_manager() as f: (1)
    ... (2)

# end of with statement scope
... (3)
```
(1) create a context manager object with `context_manager()`. `__enter__()` will be invoked and whatever value returned will be assigned to `f`<br>
(2) statements run inside our **context**<br>
(3) right after the context is exited, `__exit__()` was called then statements outside **context** will be run

`__enter__(self)` **doesn't** needs to return a `context_manager` object. However, the returned value could be assigned to an itermediate variable which is accessible inside the scope of `with` statement and event **outside** of the scope, i.e. the variable is still accessible after exiting the `with` scope.<br>
It's ensured that `__exit__()` will be called, even in case of exception.

In [6]:
class context_manager:
    
    def __init__(self):
        print('Initiating a context')
        
    def __enter__(self):
        print('Entering context')
        
        return self
    
    def __exit__(*args, **kwargs):
        print('Exiting context')
        
        return None
    
with context_manager() as ctx:
    print('Inside a context with object:', ctx)
    
print('Outside a context with object:', ctx)

Initiating a context
Entering context
Inside a context with object: <__main__.context_manager object at 0x000001C463754A90>
Exiting context
Outside a context with object: <__main__.context_manager object at 0x000001C463754A90>


In [12]:
class context_manager:
    
    def __init__(self):
        print('Initiating a context')
        
    def __enter__(self):
        print('Entering context')
        
        return None  # __enter__ doesn't necessarily return any value here.
    
    def __exit__(*args, **kwargs):
        print('Exiting context with called args:', args, kwargs)
        
        return None
    
with context_manager() as ctx:
    print('Inside a context with object:', ctx)
    
print('Outside a context with object:', ctx)

Initiating a context
Entering context
Inside a context with object: None
Exiting context with called args: (<__main__.context_manager object at 0x000001C46376EC70>, None, None, None) {}
Outside a context with object: None


Technically, `__enter__()` doesn't need to return any value but this is exactly how `context_manager` shines.

# Use cases
1. Temporarily create a "context" which is short-lived and to be cleaned up afterward
    > A classic and most prequently used context manager is `open()` function. You open a file, do something with it and need to make sure the file will be closed after you're done with all file writing/reading operations.
    ```python
    with open('myfile.txt') as f:
        f.write('content')
    # you could be sure that myfile.txt is properly closed, even you didn't explicitly callf.closed()
    ```

    > Another useful example is to temporily create a test table in a database, do something with it and drop the table after you're done as per below implementation
        

In [21]:
import sqlite3

class database:
    
    def __init__(self, test_table_name):
        print('Initiate a sqlite3 database')
        self.connection = sqlite3.connect(":memory:")  # create a new connection to an in-memory database
        self.test_table_name = test_table_name
        
    def __enter__(self):
        # Upon entering context, create the test_table_name
        sql = f'''create table {self.test_table_name}
        (NAME TEXT NOT NULL,
        AGE INT NOT NULL
        )
        '''
        self.connection.execute(sql)
        
        print(f'table {self.test_table_name} created successfully')
        
        return self.connection  # return the connection to interact with our db
    
    def __exit__(self, *args, **kwargs):
        sql = f'drop table {self.test_table_name}'
        self.connection.execute(sql)
        
        print(f'table {self.test_table_name} is dropped successfully')
        
        return None
    
with database('my_table') as conn:
    conn.execute('Insert into my_table (Name, age) values ("An", 32)')
    conn.execute('Insert into my_table (Name, age) values ("Vien", 32)')
    records = conn.execute('Select * from my_table')
    for record in records:
        print(record)
        
records = conn.execute('Select * from my_table')
for record in records:
    print(record)

Initiate a sqlite3 database
table my_table created successfully
('An', 32)
('Vien', 32)
table my_table is dropped successfully


OperationalError: no such table: my_table

Even if there is an exception during execution within `with` statement, the `__exit__()` method will be called to drop the test table.

In [30]:
import sqlite3

class database:
    
    def __init__(self, test_table_name):
        print('Initiate a sqlite3 database')
        self.connection = sqlite3.connect(":memory:")  # create a new connection to an in-memory database
        self.test_table_name = test_table_name
        
    def __enter__(self):
        # Upon entering context, create the test_table_name
        sql = f'''create table {self.test_table_name}
        (NAME TEXT NOT NULL,
        AGE INT NOT NULL
        )
        '''
        self.connection.execute(sql)
        
        print(f'table {self.test_table_name} created successfully')
        
        return self.connection  # return the connection to interact with our db
    
    def __exit__(self, *args, **kwargs):
        sql = f'drop table {self.test_table_name}'
        self.connection.execute(sql)
        
        print(f'table {self.test_table_name} is dropped successfully')
        
        return None

try:
    with database('my_table') as conn:
        conn.execute('Insert into my_table (Name, age) values ("An", 32)')
        conn.execute('Insert into my_table (Name, age) values ("Vien", 32)')

        # Raise an arbitrarily exception
        raise Exception("Something went wrong")

        records = conn.execute('Select * from my_table')
        for record in records:
            print(record)
except Exception as e:
    print(e)
        
records = conn.execute('Select * from my_table')
for record in records:
    print(record)

Initiate a sqlite3 database
table my_table created successfully
table my_table is dropped successfully
Something went wrong


OperationalError: no such table: my_table

In [37]:
class ContextDecorator:
    
    def __call__(self, func):
        if func:
            print('Cooking the function:', func)
            def wrapped_ctx(*args, **kwargs):
                with MyCtxMng() as ctx:
                    func(*args, **kwargs)

            return wrapped_ctx
        
class MyCtxMng(ContextDecorator):
    
    def __init__(self):
        print('Init MyCtxMng')
        
    
    def __enter__(self):
        print('Entering context')
        
    def __exit__(self, *args, **kwargs):
        print('Exiting context')
        
@MyCtxMng()
def myfunc(msg):
    print(msg)
    
myfunc('Hello there')
print('\n')
with MyCtxMng():
    print('Hello')

Init MyCtxMng
Cooking the function: <function myfunc at 0x000001C4649FF700>
Init MyCtxMng
Entering context
Hello there
Exiting context


Init MyCtxMng
Entering context
Hello
Exiting context


# contextlib
## contextmanager
This is a decorator that will turn a generator into a context manager when used in `with` block.

In [2]:
from contextlib import contextmanager

@contextmanager
def simple_generator():
    print('Enter Context')
    yield
    print('Exit context')
    
with simple_generator() as a:
    print('Do something')

Enter Context
Do something
Exit context


The implementation is roughly as below

In [10]:
def contextmanager(gen):
    
    # Inside the decorator, create a context manager class
    # and store the new instance of our generator
    class ctxmanager:
        
        def __init__(self):
            self.gen = gen()
            
        def __enter__(self):
            next(self.gen)
            
        def __exit__(self, *args):
            try:
                next(self.gen)
            except StopIteration:
                pass
            
    return ctxmanager

@contextmanager
def simple_generator():
    print('Enter Context')
    yield
    print('Exit context')

cm = simple_generator()

with cm:
    print('Do something')
with cm:
    print('Do something')

Enter Context
Do something
Exit context


StopIteration: 

# TODO
1. `singleuse`, `reusable`, `reentrant` context manager
2. Supporting a variable number of context manager
3. Handling exception from and inside `__enter__()` method
4. class `contextlib.ExitStack` usage
5. Real world use cases

## singleuse context manager
Refer to those context manager that can use only 1 time. This usually a generator-based. Since a generator will be exhausted (hitting StopIteration) after first use, every time you tried to get into the context which, in turn, invokes `next(generator)` it will raise `StopIteration` again.

## reentrant context manager
Is type of context manager that could be used more than one time and used within a nested `with` block of its self.

In [18]:
class my_context:
    
    number = 1
    
    def __init__(self):
        self.no = my_context.number
        my_context.number += 1
    
    def __enter__(self):
        print('Enter context', self.no)
        
    def __exit__(self, *args):
        print('Exit context', self.no)

with my_context() as ctx:
    print('Inside first context')
    with my_context() as ctx:
        print('Inside second context')
        
    print('Outside of sendcond context, inside first context')

print('Outside of first context')

Enter context 1
Inside first context
Enter context 2
Inside second context
Exit context 2
Outside of sendcond context, inside first context
Exit context 1
Outside of first context


## reusable context manager
Most of context managers are reusable, but by saying reusable here, it strickly mean that the context manager is **not reentrant**